In [1]:
from sklearn.pipeline import make_pipeline, make_union
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import roc_auc_score
from pathlib import Path

import pandas as pd

In [2]:
import sys
sys.path.append('/Users/przivic/prog/machine_learning_practico')

In [3]:
from lib import data, transformers
from lib.model import get_features_pipe, get_model_pipe

In [4]:
PATH = Path('../../data/')
movies_df = data.load_data(PATH)

Loading title basics...


/Users/przivic/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3343: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


Loading title ratings...
Loading movie directors...
Merging everything...


In [ ]:
principals_df = pd.read_csv(PATH / 'title.principals.tsv', sep='\t')

In [ ]:
principals_df.head()

In [ ]:
principals_df.category.value_counts()

In [ ]:
# Sacado del codigo de directores

movies_stars = principals_df[principals_df.category.isin(['actress', 'actor'])].copy()

# Calculo un ranking por pelicula segun el ordering
movies_stars['star_rank'] = (
    movies_stars.sort_values('ordering')
        .groupby('tconst')
        .cumcount()
)

# Me quedo con el "director principal" por pelicula
first_star = movies_stars[movies_stars.star_rank == 0][['nconst', 'tconst']].rename(columns={'nconst': '1st_star'})
second_star = movies_stars[movies_stars.star_rank == 1][['nconst', 'tconst']].rename(columns={'nconst': '2nd_star'})
third_star = movies_stars[movies_stars.star_rank == 2][['nconst', 'tconst']].rename(columns={'nconst': '3rd_star'})

In [ ]:
stars_df = first_star.merge(second_star, how='left', on='tconst').merge(third_star, how='left', on='tconst')

In [ ]:
stars_df.head()

In [ ]:
stars_df[stars_df.tconst == 'tt0120338']

In [ ]:
movies_df = movies_df.merge(stars_df, on='tconst', how='left')

In [ ]:
movies_df.head()

# Ahora  vamos a experimentar!

Como podemos hacer para usar 1st_star y 2nd_star con el código que **ya** tenemos? [Miremos el diff](https://github.com/elsonidoq/machine_learning_practico/commit/1244da3daee2f7aff140d202885e6e8dba55c099)

In [ ]:
rating_data = data.load_rating_train_dev_test(movies_df)

In [ ]:
pipe = make_pipeline(
    transformers.CrewFeatures('1st_star', min_cnt_movies=3),
    DictVectorizer(sparse=False),
    StandardScaler(),
    LogisticRegression()
)

In [ ]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

In [ ]:
pipe = make_pipeline(
    make_union(
        make_pipeline(director_features.CrewFeatures('1st_star', min_cnt_movies=5), DictVectorizer(sparse=False)),
        make_pipeline(director_features.CrewFeatures('2nd_star', min_cnt_movies=5), DictVectorizer(sparse=False)),
        make_pipeline(director_features.CrewFeatures('3rd_star', min_cnt_movies=5), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression()
)

In [ ]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc

# Probando todo junto

In [ ]:
pipe = make_pipeline(
    make_union(
        make_pipeline(transformers.CrewFeatures('1st_star', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('2nd_star', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('3rd_star', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.CrewFeatures('director', min_cnt_movies=6), DictVectorizer(sparse=False)),
        make_pipeline(transformers.YearsAgo(), DictVectorizer(sparse=False)),
        make_pipeline(transformers.GenreDummies(), DictVectorizer(sparse=False)),
    ),
    StandardScaler(),
    LogisticRegression(C=47, penalty='l1', solver='saga')
)

In [ ]:
pipe.fit(rating_data['X_train'], rating_data['y_train'] > 7.5)

tr_auc = roc_auc_score(rating_data['y_train'] > 7.5, pipe.predict_proba(rating_data['X_train'])[:, 1])
dev_auc = roc_auc_score(rating_data['y_dev'] > 7.5, pipe.predict_proba(rating_data['X_dev'])[:, 1])

tr_auc, dev_auc